<h1><center> NASA Airathon - NO2 Track </center></h1>

### <center> Single day: extract GFS data for a specific location </center>

<div style="text-align: center"> 
    Dr. Sukanta Basu <br/> Associate Professor <br/> Delft University of Technology, The Netherlands <br/> Email: s.basu@tudelft.nl<br/> https://sites.google.com/view/sukantabasu/
</div>

#### Log

Last updated: 4th April, 2022

#### User instructions 

1. Provide the latitude (LATx) and longitude (LONx) of the location of interest 

2. Provide the directory name containing GFS data for a specific date. 

3. This notebook will extract GFS data for the location of interest and save it in testGFS.csv file inside data/singleday/processed/test/GFS/ directory. 

As an illustrative example, we use the centroid of the ZZ8JF grid (longitude: -117.3274620089189, latitude: 33.664840117597805). This information is available from the data/airathon/processed/grid_latlon.csv file. 

Also, we select August 24, 2021 as our case study date. 

#### Load packages

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from pathlib import Path
import netCDF4

#### Directories

In [2]:
ROOT_DIR    = '../../'

#Only one day data is included in the repo_sukantabasu
GFS_DIR     = ROOT_DIR + 'data/singleday/raw/GFS/'

#Location of processed datasets
EXTDATA_DIR = ROOT_DIR + 'data/singleday/processed/'

#### Provide coordinates

In [3]:
LATx = 33.664840117597805
LONx = -117.3274620089189

#### Provide date

In [4]:
YR = 2021
MO = 8
DY = 24
HR = 0

#### Compute Julian day and week day

In [5]:
df = pd.DataFrame({'year': [YR],
                   'month': [MO],
                   'day': [DY]})
JD = (pd.to_datetime(df)).dt.dayofyear
WD = (pd.to_datetime(df)).dt.weekday

#### Compute nearest grid point

In [6]:
def nearestGridFAST(LAT,LON,LATx,LONx):
    
    dLAT = np.abs(LAT - LATx)
    dLON = np.abs(LON - LONx)
    
    dTOT = dLAT + dLON #taxi-cab distance
    
    #Interesting function; https://stackoverflow.com/questions/3230067/numpy-minimum-in-row-column-format
    r_min, c_min = np.unravel_index(dTOT.argmin(), dTOT.shape)
    
    return r_min, c_min

#### Calculate required forecast horizon based on location

In [7]:
if (LONx > 120) & (LONx < 122):
    strCITY = 'TAI/'
    fcstHRini = 15
    fcstHRfin = 39+1
elif (LONx > 76) & (LONx < 78):
    strCITY = 'DEL/'
    fcstHRini = 18
    fcstHRfin = 42+1
elif (LONx > -119) & (LONx < -116):
    strCITY = 'LAX/'
    fcstHRini = 6
    fcstHRfin = 30+1        
else:
    strCITY = 'XXX/'

print(strCITY)

LAX/


#### Extract GFS data

In [8]:
GFSdata = np.zeros((1,13*9)) #13 variables (PBLH, M10, M100, alpha, sinX100, cosX100, 
                                    #beta, SHFX, VENT, RH, T2, T100-T2, Rig); 
                                    #24 h = 9 x 3-hourly samples spanning 0 h to 0 h

cnt = 0
for fcstHR in np.arange(fcstHRini,fcstHRfin,3): #I use data from 3 UTC to 24 UTC

    strFILE = 'gfs.0p25.' + str(YR) + str(MO).zfill(2) + str(DY).zfill(2) + str(HR).zfill(2) + '.f' + str(fcstHR).zfill(3) + '.grib2.nc'

    #-----------------
    #TMP, U GRD, V GRD, RH are stored here
    fileGFS_VAR1 = Path(GFS_DIR + 'VAR1/' + strFILE)
    dataGFS_VAR1 = netCDF4.Dataset(fileGFS_VAR1,'r')

    #PBLH, SHFX, VRATE are stored here
    fileGFS_VAR2 = Path(GFS_DIR + 'VAR2/' + strFILE)
    dataGFS_VAR2 = netCDF4.Dataset(fileGFS_VAR2,'r')
    #-----------------

    #-----------------
    LAT  = dataGFS_VAR1.variables['lat'][:]
    LON  = dataGFS_VAR1.variables['lon'][:]
    if (strCITY == 'LAX/'): 
        LON = LON - 360 #GFS outputs data as 240 E instead of -120 W
    LON2d, LAT2d = np.meshgrid(LON,LAT)

    r_min, c_min = nearestGridFAST(LAT2d,LON2d,LATx,LONx)
    #-----------------

    #PBL height
    PBLH = np.squeeze(dataGFS_VAR2.variables['HPBL_L1'][:])
    PBLHi = PBLH[r_min][c_min]
    GFSdata[0,cnt] = PBLHi
    cnt = cnt + 1
    
    #U component of velocity
    dum  = np.squeeze(dataGFS_VAR1.variables['U_GRD_L103'][:])
    U100 = np.squeeze(dum[0,:,:])
    U10  = np.squeeze(dum[1,:,:])
    U10i = U10[r_min][c_min]
    U100i= U100[r_min][c_min]
    
    #V component of velocity
    dum  = np.squeeze(dataGFS_VAR1.variables['V_GRD_L103'][:])
    V100 = np.squeeze(dum[0,:,:])
    V10  = np.squeeze(dum[1,:,:])
    V10i = V10[r_min][c_min]
    V100i= V100[r_min][c_min]
    
    #Wind speed and direction
    M10i      = np.sqrt(U10i**2  + V10i**2)
    M100i     = np.sqrt(U100i**2 + V100i**2)
    alpha     = np.log(M100i/M10i)/np.log(10)

    RtoD      = 180/np.pi
    X10i      = np.arctan2(-U10i,-V10i)*RtoD
    if X10i < 0:
        X10i = X10i + 360
    X100i     = np.arctan2(-U100i,-V100i)*RtoD
    if X100i < 0:
        X100i = X100i + 360
    sinX100i  = np.sin(X100i/RtoD)
    cosX100i  = np.cos(X100i/RtoD)
    beta      = np.abs(X100i-X10i)
    if beta > 180:
        beta = 360 - beta

    GFSdata[0,cnt] = M10i
    cnt = cnt + 1

    GFSdata[0,cnt] = M100i
    cnt = cnt + 1

    GFSdata[0,cnt] = alpha 
    cnt = cnt + 1

    GFSdata[0,cnt] = sinX100i 
    cnt = cnt + 1

    GFSdata[0,cnt] = cosX100i 
    cnt = cnt + 1

    GFSdata[0,cnt] = beta 
    cnt = cnt + 1
    
    #Sensible heat flux
    SHFX  = np.squeeze(dataGFS_VAR2.variables['SHTFL_L1_Avg_1'][:])
    SHFXi = SHFX[r_min][c_min]
    GFSdata[0,cnt] = SHFXi
    cnt = cnt + 1
    
    #Ventillation rate
    VENT = np.squeeze(dataGFS_VAR2.variables['VRATE_L220'][:])
    VENTi = VENT[r_min][c_min]
    GFSdata[0,cnt] = VENTi
    cnt = cnt + 1    
    
    #Relative humidity
    RH   = np.squeeze(dataGFS_VAR1.variables['R_H_L103'][:])
    RHi   = RH[r_min][c_min]
    GFSdata[0,cnt] = RHi
    cnt = cnt + 1

    #Air temperature 
    dum  = np.squeeze(dataGFS_VAR1.variables['TMP_L103'][:])
    T100 = np.squeeze(dum[0,:,:])
    T2   = np.squeeze(dum[1,:,:])
    T100i= T100[r_min][c_min]
    T2i  = T2[r_min][c_min]
    GFSdata[0,cnt] = T2i
    cnt = cnt + 1
    
    dT   = T100 - T2
    dTi   = dT[r_min][c_min]
    GFSdata[0,cnt] = dTi
    cnt = cnt + 1
    
    #Gradient Richardson Number
    GFSdata[0,cnt] = (T100i - T2i)/(M100i - M10i)**2
    cnt = cnt + 1                

    #-----------------

print(GFSdata)

[[ 1.06158953e+01  7.83058817e-01  6.47857207e-01 -8.23150893e-02
   9.84193997e-01 -1.77093693e-01  2.91373206e+01  1.77946777e+01
   0.00000000e+00  8.03000031e+01  2.89831482e+02  2.58959961e+00
   1.41667065e+02  1.06545696e+01  4.91800276e-01  6.22303298e-01
   1.02213335e-01  5.88641393e-01  8.08394279e-01  2.03624673e+00
  -2.00932622e+00  0.00000000e+00  8.40000000e+01  2.88092865e+02
   3.43923950e+00  2.01939503e+02  1.06647186e+01  4.39861571e-01
   6.60091954e-01  1.76288418e-01  8.03931885e-01  5.94721384e-01
   1.65030006e+01 -1.54204106e+00  0.00000000e+00  8.38000031e+01
   2.87260101e+02  4.19110107e+00  8.64119219e+01  1.44277786e+02
   5.68845488e-01  5.49083747e-01 -1.53557289e-02  7.88129369e-01
   6.15509624e-01  1.02825732e+01  8.96738243e+00  0.00000000e+00
   5.77000008e+01  2.93494659e+02 -1.16729736e+00 -2.98903572e+03
   9.85858521e+02  1.63897369e+00  1.92926067e+00  7.08189271e-02
  -1.08523463e-01 -9.94093888e-01  4.30833306e+00  1.14826683e+02
   2.00000

In [9]:
#### Save GFS data in a csv file

In [10]:
df_new = pd.DataFrame(data=GFSdata)
df_new.columns = ['PBLH_0' ,'M10_0' ,'M100_0' ,'alpha_0', 'sinX100_0', 'cosX100_0', 
                  'beta_0', 'SHFX_0', 'VENT_0', 'RH_0', 'T2_0', 'dT_0', 'Rig_0',
                  'PBLH_3' ,'M10_3' ,'M100_3' ,'alpha_3', 'sinX100_3', 'cosX100_3', 
                  'beta_3', 'SHFX_3', 'VENT_3', 'RH_3', 'T2_3', 'dT_3', 'Rig_3',
                  'PBLH_6' ,'M10_6' ,'M100_6' ,'alpha_6', 'sinX100_6', 'cosX100_6', 
                  'beta_6', 'SHFX_6', 'VENT_6', 'RH_6', 'T2_6', 'dT_6', 'Rig_6',
                  'PBLH_9' ,'M10_9' ,'M100_9' ,'alpha_9', 'sinX100_9', 'cosX100_9', 
                  'beta_9', 'SHFX_9', 'VENT_9', 'RH_9', 'T2_9', 'dT_9', 'Rig_9',
                  'PBLH_12','M10_12','M100_12','alpha_12','sinX100_12','cosX100_12',
                  'beta_12','SHFX_12','VENT_12','RH_12','T2_12','dT_12','Rig_12',
                  'PBLH_15','M10_15','M100_15','alpha_15','sinX100_15','cosX100_15',
                  'beta_15','SHFX_15','VENT_15','RH_15','T2_15','dT_15','Rig_15',
                  'PBLH_18','M10_18','M100_18','alpha_18','sinX100_18','cosX100_18',
                  'beta_18','SHFX_18','VENT_18','RH_18','T2_18','dT_18', 'Rig_18',
                  'PBLH_21','M10_21','M100_21','alpha_21','sinX100_21','cosX100_21',
                  'beta_21','SHFX_21','VENT_21','RH_21','T2_21','dT_21', 'Rig_21',
                  'PBLH_24','M10_24','M100_24','alpha_24','sinX100_24','cosX100_24',
                  'beta_24','SHFX_24','VENT_24','RH_24','T2_24','dT_24', 'Rig_24']       
    
df_new.insert(0, 'WDAY', WD)
df_new.insert(1, 'sinJDAY', np.sin(2*np.pi*JD/365))
df_new.insert(2, 'cosJDAY', np.cos(2*np.pi*JD/365))

print(df_new)

df_new.to_csv(EXTDATA_DIR + 'test/GFS/' + 'testGFS.csv', index=False)

   WDAY   sinJDAY   cosJDAY     PBLH_0     M10_0    M100_0   alpha_0  \
0     1 -0.796183 -0.605056  10.615895  0.783059  0.647857 -0.082315   

   sinX100_0  cosX100_0     beta_0  ...  alpha_24  sinX100_24  cosX100_24  \
0   0.984194  -0.177094  29.137321  ...  0.199829    0.686846    0.726803   

     beta_24    SHFX_24  VENT_24      RH_24       T2_24    dT_24      Rig_24  
0  28.437592  12.656713      0.0  54.299999  291.958618  4.04776  131.094604  

[1 rows x 120 columns]
